In [ ]:
!pip install rasa==1.10.3

In [ ]:
!python -m spacy download en

In [ ]:
!pip install nest_asyncio==1.3.3

In [4]:
import os
import rasa
import nest_asyncio

nest_asyncio.apply()

print("event loop ready")

event loop ready


In [5]:
from rasa.cli.scaffold import create_initial_project

In [6]:
project='test_project'
create_initial_project(project)

In [7]:
os.chdir(project)
print(os.listdir("."))

['config.yml', 'actions.py', '__pycache__', 'tests', 'endpoints.yml', '__init__.py', 'data', 'domain.yml', 'credentials.yml']


In [8]:
config = "config.yml"
training_files = "data/"
domain = 'domain.yml'
output = 'models/'
print( config, training_files, domain, output)

config.yml data/ domain.yml models/


In [10]:
%%writefile data/nlu.md

## intent:clean_now
- can you send someone right now
- someone available now
- right now please
- please clean it right now

## intent:clean
- can you send someone to clean my Room
- room service is available
- i need to clean my room
- room cleaning
- clean my room

## intent:deluxe
- i need a deluxe room
- deluxe room
- i am looking for deluxe room
- i want deluxe room
- can i get the deluxe room

## intent:simple
- i need a simple room
- simple room
- i am looking for simple room
- i want simple room
- can i get the simple room

## intent:num_room
- i need [2](number) rooms
- i need [3](number) rooms
- i need whole floor
- just [1](number) room
- [2](number) rooms
- only [2](number) rooms
- [1](number) room

## intent:book_room
- i want to book a room
- i need a room
- can i get a room
- room is available
- i want a room
- book a room
- is room available

## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?


Overwriting data/nlu.md


In [11]:
%%writefile domain.yml

intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge
  - book_room
  - num_room
  - simple
  - deluxe
  - clean
  - clean_now

responses:

  utter_clean_now:
  - text: "sure, i am sending someone to clean your room"

  utter_clean:
  - text: "when should i arrange for the cleaning?"

  utter_deluxe:
  - text: "you have choosen to book a deluxe room."

  utter_simple:
  - text: "you have choosen to book a simple room."

  utter_num_room:
  - text: "What Kind of Room would you want to book - simple room or deluxe room ?"

  utter_book_room:
  - text: "How many room would you like to book ?"

  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great, carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_iamabot:
  - text: "I am a bot, powered by Rasa."

entities:
  - number
session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true


Overwriting domain.yml


In [12]:
%%writefile data/stories.md

## clean my room
* greet
  - utter_greet
* clean
  - utter_clean
* clean_now
  - utter_clean_now

## booking simple room
* greet
  - utter_greet
* book_room
  - utter_book_room
* num_room
  - utter_num_room
* simple
  - utter_simple 

## booking deluxe room
* greet
  - utter_greet
* book_room
  - utter_book_room
* num_room
  - utter_num_room
* deluxe
  - utter_deluxe

## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

## bot challenge
* bot_challenge
  - utter_iamabot


Overwriting data/stories.md


In [16]:
model_path = rasa.train(domain,config, [training_files], output)
print(model_path)

Training Core model...


Processed trackers: 100%|██████████| 8/8 [00:00<00:00, 85.17it/s, # actions=28]
Processed actions: 28it [00:00, 470.44it/s, # examples=28]
Epochs:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/rasa/utils/tensorflow/model_data.py:386: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  final_data[k].append(np.concatenate(np.array(v)))
Epochs: 100%|██████████| 100/100 [00:25<00:00,  3.88it/s, t_loss=0.094, loss=0.007, acc=1.000]


Core model training completed.
Training NLU model...


/usr/local/lib/python3.7/dist-packages/rasa/nlu/classifiers/diet_classifier.py:595: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for tag_name, tag_ids in tag_name_to_tag_ids.items()
Epochs:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/rasa/utils/tensorflow/model_data.py:386: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  final_data[k].append(np.concatenate(np.array(v)))
Epochs: 100%|██████████| 100/100 [00:27<00:00,  3.58it/s, t_loss=2.493, i_loss=0.874, entity_loss=0.013, i_acc=1.000, entity_f1=0.903]


NLU model training completed.
Your Rasa model is trained and saved at '/content/test_project/models/20220204-102958.tar.gz'.
models/20220204-102958.tar.gz


In [17]:
from rasa.jupyter import chat

endpoints = 'endpoints.yml'

chat(model_path, endpoints)

Instructions for updating:
Use tf.identity instead.
Your bot is ready to talk! Type your messages here or send '/stop'.
hi
Hey! How are you?
i need a room
How many room would you like to book ?
2 rooms
What Kind of Room would you want to book - simple room or deluxe room ?
deluxe room
you have choosen to book a deluxe room.
/stop
